<a href="https://colab.research.google.com/github/manthanawgan/MLops/blob/main/Duration_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.__version__

'2.2.2'

In [13]:
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet")

In [43]:
df.shape

(3403248, 21)

In [44]:
df.dtypes

,0
VendorID,int32
tpep_pickup_datetime,datetime64[us]
tpep_dropoff_datetime,datetime64[us]
passenger_count,float64
trip_distance,float64
RatecodeID,float64
store_and_fwd_flag,object
PULocationID,int32
DOLocationID,int32
payment_type,int64


In [20]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td : td.total_seconds() / 60)

In [33]:
df.duration.describe(percentiles = [0.99, 0.98, 0.95] )

,duration
count,3.475226e+06
mean,1.501812e+01
std,3.871358e+01
min,-5.147232e+04
50%,1.170000e+01
95%,3.620000e+01
98%,4.931667e+01
99%,5.891667e+01
max,5.626317e+03


In [36]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [50]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

df[categorical].dtypes

,0
PULocationID,object
DOLocationID,object
